# Transaction Data Processing

This notebook processes transaction data with optimized memory management and handles data quality issues.

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from glob import glob
import gc
from datetime import datetime
import warnings
import psutil  # For memory monitoring
from tqdm import tqdm  # For progress bars (install with pip install tqdm if needed)
import time

In [3]:
def get_memory_usage():
    """Return the current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)

def log_memory(message):
    """Log memory usage with a custom message"""
    print(f"{message}: {get_memory_usage():.2f} MB")

## 1. Data Cleaning Functions

Define functions for cleaning and feature engineering before we use them

In [4]:
def clean_chunk(df):
    """Clean a chunk of transaction data with optimized operations"""
    # Make a copy only if needed (can improve performance but may cause SettingWithCopyWarning)
    df = df.copy()
    
    # Handle string columns efficiently by processing only non-null values
    str_cols = ['merchant_name', 'merchant_category', 'currency', 'location_country', 'location_city', 'device_type', 'status']
    for col in str_cols:
        if col in df.columns:
            if df[col].dtype == 'string' or df[col].dtype == 'object':
                # Only process non-null values to save time
                mask = df[col].notna()
                if mask.any():
                    df.loc[mask, col] = df.loc[mask, col].str.strip().str.lower()
            
            # Handle categorical columns properly before filling with 'unknown'
            if pd.api.types.is_categorical_dtype(df[col]):
                # Add 'unknown' to the category list if it's not already there
                if 'unknown' not in df[col].cat.categories:
                    df[col] = df[col].cat.add_categories(['unknown'])
            
            # Fill missing values
            df[col] = df[col].fillna('unknown')
    
    # Handle missing numeric values more efficiently
    if 'amount' in df.columns:
        df['missing_amount'] = df['amount'].isna()
        df['amount'] = df['amount'].fillna(0)
    
    # Handle missing boolean values
    if 'is_online' in df.columns:
        df['is_online'] = df['is_online'].fillna(False)
    
    # Handle outliers in amount using vectorized operations
    if 'amount' in df.columns:
        # Use robust statistics to avoid influence of extreme outliers
        q1 = df['amount'].quantile(0.25)
        q3 = df['amount'].quantile(0.75)
        iqr = q3 - q1
        upper_limit = q3 + 1.5 * iqr
        df['is_amount_outlier'] = df['amount'] > upper_limit
    
    # Drop duplicates efficiently using subset
    df.drop_duplicates(subset='transaction_id', keep='first', inplace=True)
    
    return df

## 2. Feature Engineering Functions

In [5]:
def add_derived_features(df):
    """Add derived features to the dataframe with minimal memory impact"""
    # Ensure timestamp is a datetime object
    if 'timestamp' in df.columns:
        # Make sure timestamp is datetime type
        if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
        
        # Extract time-based features efficiently
        df['day_of_week'] = df['timestamp'].dt.dayofweek
        df['is_weekend'] = df['day_of_week'].isin([5, 6])
        df['hour'] = df['timestamp'].dt.hour
        
        # Time period categorization
        conditions = [
            (df['hour'] < 6),
            (df['hour'] >= 6) & (df['hour'] < 12),
            (df['hour'] >= 12) & (df['hour'] < 18),
            (df['hour'] >= 18)
        ]
        choices = ['night', 'morning', 'afternoon', 'evening']
        df['time_period'] = np.select(conditions, choices, default='unknown')
        
        # Calculate days since transaction efficiently
        current_date = pd.Timestamp(datetime.now().date())
        df['days_since_transaction'] = (current_date - df['timestamp'].dt.floor('D')).dt.days
    
    # Flag potential anomalies using vectorized operations
    if 'amount' in df.columns and 'is_online' in df.columns:
        df['high_value_online'] = (df['amount'] > 1000) & df['is_online']
    
    return df

## 3. Memory-Efficient Processing Function

In [6]:
def process_transactions(file_list, chunksize=100000, output_dir="processed_chunks", max_files=None):
    """
    Process transaction files in chunks with improved memory management and proper deduplication
    """

    # Import required modules
    import os
    import gc
    import pandas as pd
    import numpy as np
    import time
    from tqdm import tqdm
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Limit number of files if specified
    if max_files is not None and max_files < len(file_list):
        print(f"Limiting to {max_files} out of {len(file_list)} files")
        file_list = file_list[:max_files]
    
    # Track processing statistics
    total_rows_processed = 0
    total_chunks_processed = 0
    start_time = time.time()
    output_files = []
    
    # IMPROVED APPROACH: Create a single set of transaction IDs that we'll update as we process each file
    # This way we'll only keep track of transactions we've already processed
    seen_transaction_ids = set()
    
    # Process each file and save to parquet
    print("\n--- Processing and saving files ---")
    
    for i, file_path in enumerate(file_list):
        file_start_time = time.time()
        print(f"\nProcessing file {i+1}/{len(file_list)}: {os.path.basename(os.path.dirname(file_path))}")
        log_memory(f"Memory before processing file {i+1}")
        
        try:
            # Get file size for progress reporting
            file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
            print(f"File size: {file_size:.2f} MB")
            
            # Calculate expected chunks
            with open(file_path, 'r') as f:
                approx_rows = sum(1 for _ in f) - 1  # Subtract header
            print(f"Approximate rows: {approx_rows:,}")
            expected_chunks = approx_rows // chunksize + (1 if approx_rows % chunksize else 0)
            
            # Create output file path for this file
            output_file = os.path.join(output_dir, f"processed_file_{i+1}.parquet")
            
            # Process current file in chunks
            chunk_reader = pd.read_csv(
                file_path, 
                dtype=dtypes,
                parse_dates=['timestamp'],
                chunksize=chunksize
            )
            
            # Initialize a buffer for this file's processed chunks
            processed_chunks = []
            file_rows_processed = 0
            
            # Process each chunk
            for chunk_idx, chunk in enumerate(tqdm(chunk_reader, total=expected_chunks, desc=f"Processing file {i+1}")):
                # Convert timestamp if needed
                if not pd.api.types.is_datetime64_any_dtype(chunk['timestamp']):
                    chunk['timestamp'] = pd.to_datetime(chunk['timestamp'], errors='coerce')
                
                # Clean data
                chunk = clean_chunk(chunk)
                
                # IMPROVED: Filter out only transaction IDs we've already seen
                # Get the current chunk's transaction IDs
                current_ids = set(chunk['transaction_id'])
                
                # Find which transaction IDs in this chunk are duplicates (already seen)
                duplicate_ids = current_ids.intersection(seen_transaction_ids)
                
                # Remove duplicates if any found
                if duplicate_ids:
                    chunk = chunk[~chunk['transaction_id'].isin(duplicate_ids)]
                
                # Add the non-duplicate transaction IDs to our seen set
                seen_transaction_ids.update(current_ids - duplicate_ids)
                
                # Skip further processing if chunk is empty after deduplication
                if len(chunk) == 0:
                    continue
                
                # Add derived features
                chunk = add_derived_features(chunk)
                
                # Add to processed chunks
                processed_chunks.append(chunk)
                file_rows_processed += len(chunk)
                
                # Update stats
                total_rows_processed += len(chunk)
                total_chunks_processed += 1
                
                # Check memory usage, if too high, save intermediate results
                current_mem = get_memory_usage()
                if current_mem > 2000 or (chunk_idx > 0 and chunk_idx % 5 == 0):  # Save every 5 chunks or if memory high
                    # Combine processed chunks and save
                    if processed_chunks:
                        combined_df = pd.concat(processed_chunks, ignore_index=True)
                        # If first chunk set, save to new file, otherwise append
                        if not os.path.exists(output_file):
                            combined_df.to_parquet(output_file)
                        else:
                            existing_df = pd.read_parquet(output_file)
                            pd.concat([existing_df, combined_df], ignore_index=True).to_parquet(output_file)
                            del existing_df
                        
                        # Clear memory
                        del combined_df
                        processed_chunks = []
                        gc.collect()
            
            # Save any remaining chunks
            if processed_chunks:
                combined_df = pd.concat(processed_chunks, ignore_index=True)
                # If we've already written to the file, append
                if os.path.exists(output_file):
                    existing_df = pd.read_parquet(output_file)
                    pd.concat([existing_df, combined_df], ignore_index=True).to_parquet(output_file)
                    del existing_df
                else:
                    combined_df.to_parquet(output_file)
                
                # Clear memory
                del combined_df
                processed_chunks = []
                gc.collect()
            
            # Add output file to list if it exists and has data
            if os.path.exists(output_file):
                file_size = os.path.getsize(output_file) / (1024 * 1024)  # Size in MB
                print(f"Created output file: {output_file} ({file_size:.2f} MB)")
                output_files.append(output_file)
                print(f"Rows processed in this file: {file_rows_processed:,}")
            
            file_time = time.time() - file_start_time
            print(f"File {i+1} processed in {file_time:.2f} seconds")
            
            # Periodic garbage collection
            gc.collect()
            
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            import traceback
            traceback.print_exc()
    
    # Report overall statistics
    total_time = time.time() - start_time
    rows_per_second = total_rows_processed / total_time if total_time > 0 else 0
    print(f"\nProcessing complete!")
    print(f"Total rows processed: {total_rows_processed:,}")
    print(f"Total chunks processed: {total_chunks_processed:,}")
    print(f"Total output files: {len(output_files)}")
    print(f"Total processing time: {total_time:.2f} seconds")
    print(f"Processing speed: {rows_per_second:.2f} rows per second")
    
    # Return the list of output files
    return output_files

## 4. Configure Data Loading

In [8]:
# Define optimized dtypes
dtypes = {
    'transaction_id': 'string',
    'user_id': 'string',
    'merchant_id': 'string',
    'merchant_name': 'string',
    'merchant_category': 'category',
    'amount': 'float32',
    'currency': 'category',
    'location_country': 'category',
    'location_city': 'category',
    'is_online': 'bool',
    'device_type': 'category',
    'status': 'category'
}

# Find all transaction files
base_dir = 'transaction_data'
all_csv_files = glob(os.path.join(base_dir, '**', 'transactions.csv'), recursive=True)
print(f"Found {len(all_csv_files)} transaction files")

Found 31 transaction files


## 5. Process the Data

In [9]:
SAMPLE_FRACTION = None # Process 5% of the data to build a representative sample
MAX_FILES = None  # Limit to first N files for initial analysis
CHUNK_SIZE = 100000  # Adjust based on available RAM

In [10]:
print("\n=== STARTING DATA PROCESSING ===")
log_memory("Initial memory usage")

# Comment out the sampling parameters for full dataset processing once your code is working
output_files = process_transactions(
    all_csv_files, 
    chunksize=CHUNK_SIZE,
    output_dir="processed_chunks",
    max_files=MAX_FILES  # Remove this for full processing
)

log_memory("Memory usage after data loading")

# Load the processed data for analysis
if output_files:
    print(f"Successfully processed {len(output_files)} files")
    # Load the first file to get a sample for initial exploration
    sample_df = pd.read_parquet(output_files[0])
    print(f"Sample DataFrame shape: {sample_df.shape}")
    print(f"Sample DataFrame memory usage: {sample_df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
else:
    print("No output files were generated")


=== STARTING DATA PROCESSING ===
Initial memory usage: 119.88 MB

--- Processing and saving files ---

Processing file 1/31: 2025-04-07
Memory before processing file 1: 119.88 MB
File size: 245.77 MB
Approximate rows: 1,708,500


Processing file 1:   0%|          | 0/18 [00:00<?, ?it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 1:   6%|▌         | 1/18 [00:00<00:06,  2.57it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 1:  11%|█         | 2/18 [00:00<00:06,  2.50it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 1:  1

Created output file: processed_chunks\processed_file_1.parquet (81.97 MB)
Rows processed in this file: 1,708,500
File 1 processed in 16.94 seconds

Processing file 2/31: 2025-04-08
Memory before processing file 2: 1112.18 MB
File size: 245.77 MB
Approximate rows: 1,708,500


Processing file 2:   0%|          | 0/18 [00:00<?, ?it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 2:   6%|▌         | 1/18 [00:00<00:07,  2.19it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 2:  11%|█         | 2/18 [00:00<00:07,  2.18it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 2:  1

Created output file: processed_chunks\processed_file_2.parquet (81.97 MB)
Rows processed in this file: 1,708,500
File 2 processed in 18.02 seconds

Processing file 3/31: 2025-04-09
Memory before processing file 3: 1328.94 MB
File size: 245.77 MB
Approximate rows: 1,708,500


...

Created output file: processed_chunks\processed_file_29.parquet (81.96 MB)
Rows processed in this file: 1,708,500
File 29 processed in 141.20 seconds

Processing file 30/31: 2025-05-06
Memory before processing file 30: 7487.72 MB
File size: 245.77 MB
Approximate rows: 1,708,500


Processing file 30:   0%|          | 0/18 [00:00<?, ?it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 30:   6%|▌         | 1/18 [00:08<02:17,  8.07s/it]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 30:  11%|█         | 2/18 [00:16<02:13,  8.36s/it]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 30

Created output file: processed_chunks\processed_file_30.parquet (81.98 MB)
Rows processed in this file: 1,708,500
File 30 processed in 168.75 seconds

Processing file 31/31: 2025-05-07
Memory before processing file 31: 7710.64 MB
File size: 245.76 MB
Approximate rows: 1,708,500


Processing file 31:   0%|          | 0/18 [00:00<?, ?it/s]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 31:   6%|▌         | 1/18 [00:05<01:36,  5.65s/it]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 31:  11%|█         | 2/18 [00:11<01:32,  5.80s/it]C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10444\244038399.py:17: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):
Processing file 31

Created output file: processed_chunks\processed_file_31.parquet (81.98 MB)
Rows processed in this file: 1,708,500
File 31 processed in 178.93 seconds

Processing complete!
Total rows processed: 52,963,500
Total chunks processed: 558
Total output files: 31
Total processing time: 2695.49 seconds
Processing speed: 19648.93 rows per second
Memory usage after data loading: 1721.60 MB
Successfully processed 31 files
Sample DataFrame shape: (1708500, 21)
Sample DataFrame memory usage: 658.27 MB


## Data Overview